In [ ]:
# ! pip install kafka-python

In [38]:
from kafka import KafkaConsumer, TopicPartition

In [39]:
consumer1 = KafkaConsumer(bootstrap_servers='localhost:9092')

consumer1.subscribe(['mobilestreaming'])

### Sensor Data and Time Stamp

In [73]:
import mysql.connector as mysql

In [74]:
# enter your server IP address/domain name
HOST = "remotemysql.com" # or "domain.com"

# database name, if you want just to connect to MySQL server, leave it empty
DATABASE = "CA5uu0B5Nd"

# this is the user you create
USER = "CA5uu0B5Nd"

# user password
PASSWORD = "60bW7aJsjO"

In [75]:
# connect to MySQL server
db_connection = mysql.connect(host=HOST, database=DATABASE, user=USER, password=PASSWORD)
print("Connected to:", db_connection.get_server_info())

Connected to: 8.0.13-4


In [76]:
cursor = db_connection.cursor()
 
# Drop table if it already exist using execute()
cursor.execute("DROP TABLE IF EXISTS MOBILE")
 
# Create table as per requirement
sql = "CREATE TABLE MOBILE ( TIME TIMESTAMP, ACCX DOUBLE, ACCY DOUBLE, ACCZ DOUBLE, ACC DOUBLE, GYROX DOUBLE, GYROY DOUBLE, GYROZ DOUBLE, GYRO DOUBLE )"
 
cursor.execute(sql) #table created
 
# disconnect from server
# db_connection.close()

In [77]:
import numpy as np

def produce_magnitude(listt):
    magg = np.sqrt(listt[0]**2 + listt[1]**2 + listt[2]**2)
    return magg

In [81]:
def add_sql(data):
    
    db_connection = mysql.connect(host=HOST, database=DATABASE, user=USER, password=PASSWORD)
    print("Connected to:", db_connection.get_server_info())
    
    cursor = db_connection.cursor()
    
    insert_stmt = ("INSERT INTO MOBILE(TIME, ACCX, ACCY, ACCZ, ACC, GYROX, GYROY, GYROZ, GYRO)"
               "VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s)")

    row = (data["@timestamp"], 
            data["accelerometer"][0], data["accelerometer"][1], data["accelerometer"][2], 
            produce_magnitude(data["accelerometer"]),
            data["gyroscope"][0], data["gyroscope"][1], data["gyroscope"][2], 
            produce_magnitude(data["gyroscope"]))

    try:
        cursor.execute(insert_stmt, rpw)
        db_connection.commit()
        db_connection.close()

    except:
        db_connection.rollback()
        db_connection.close()

    print("Data inserted : ", row)

In [82]:
import ast
import time
 
i = 0
for msg in consumer1:
    
    i = i + 1
    
    # 100 equivalent to 7 seconds
    if i%100 == 0:
    
        byte_str = msg[6]
        dict_str = byte_str.decode("UTF-8")
        data = ast.literal_eval(dict_str)

        if "gyroscope" in data:
            print("Time Stamp : ", data["@timestamp"])
            print("Accelerometer : ", data["accelerometer"])
            print("Gyroscope : ", data["gyroscope"])
            
            add_sql(data)
            
        break

Time Stamp :  2021-04-27T12:13:19.290485
Accelerometer :  [0.29, 0.239, 9.966]
Gyroscope :  [-0.001, -0.0, 0.0]
Connected to: 8.0.13-4
Data inserted :  ('2021-04-27T12:13:19.290485', 0.29, 0.239, 9.966, 9.973082622740073, -0.001, -0.0, 0.0, 0.001)


### Sample data